**Please only run the following cell if you're running this notebook on Google colab or another cloud hosted notebook server.**

In [0]:
!pip install qiskit

# BB84 Quantum Key Distribution (QKD) Protocol

This notebook is a _demonstration_ of the BB84 Protocol for QKD using Qiskit. 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
The first three sections of the paper are readable and should give you all the necessary information required. 

[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 

$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [0]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ


from qiskit.tools.monitor import job_monitor
%pylab inline

First we will simulate a noiseless quantum channel free from an eavesdropper. The qubit capabilites of the available backends are listed below:


| Backend | Number of qubits |
|----------|------------------|
|qasm_simulator|24|
|ibmq_qasm_simulator|32|
|ibmqx4|5|
|ibmqx2|5|
|ibmq_16_melbourne|14|

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

0 $\rightarrow$ Computational basis

1  $\rightarrow$ Hadamard basis

The other encodes the state:

0  $\rightarrow \ket{ 0 } or \ket{+} $ 

1  $\rightarrow \ket{ 1 } or \ket{-} $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [0]:
num_qubits = 24 


alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(alice_state)
print(alice_basis)
print(bob_basis)

## Creating the circuit

Based on the following results:

$X\ket{0} = \ket{1}$

$H\ket{0} = \ket{+}$

$ HX\ket{0} = \ket{-}$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 



In [0]:
q = QuantumRegister(num_qubits,'q')
c = ClassicalRegister(num_qubits,'c')

bb84 = QuantumCircuit(q,c,name='bb84')

# Alice prepares her qubits
for index, _ in enumerate(alice_basis):
    if alice_state[index] == 1:
        bb84.x(q[index])
    if alice_basis[index] == 1:
        bb84.h(q[index])
bb84.barrier()  

# Bob measures the received qubits
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 1:
        bb84.h(q[index])
        
bb84.barrier()        
bb84.measure(q,c)
    
            
# bb84.draw()


### Running it on the IBMQ machines

The following cell is only needed if you want to use the IBM Q Experience devices. Please enter your API token between the quotes. Instructions are given [here](https://github.com/Qiskit/qiskit/blob/master/docs/install.rst#access-ibm-q-devices)
Uncomment the code in the cell to use it. 

In [0]:
# token = ''
# IBMQ.enable_account(token)
# IBMQ.backends()

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
# backend = IBMQ.get_backend('ibmq_qasm_simulator')

job = execute(bb84,backend,shots=1024)

job_monitor(job)

result = job.result()
counts = result.get_counts()

# Prints out all the measured binary strings by Bob (reversed)
for count in [*counts]:
    print(count) 

In [0]:
for temp_key in [*counts]:
    temp_key_rev = temp_key[::-1] # qubit orders are reversed in qiskit
    key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
             key += str(temp_key_rev[i])
    print(key)